In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
from bs4 import BeautifulSoup
import numpy as np
import requests
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
with open('vetting_playlist.json', 'r') as file:
    data1 = json.load(file)

In [3]:
#data1.keys()
#This is for someone else

In [4]:
file_paths = [
    'mpd.slice.15000-15999.json',
    'mpd.slice.16000-16999.json',
    'mpd.slice.17000-17999.json',
    'mpd.slice.18000-18999.json',
    'mpd.slice.19000-19999.json',
    'mpd.slice.20000-20999.json',
    'mpd.slice.21000-21999.json',
    'mpd.slice.22000-22999.json',
    'mpd.slice.23000-23999.json',
    'mpd.slice.24000-24999.json',
    'mpd.slice.25000-25999.json',
    'mpd.slice.26000-26999.json',
    'mpd.slice.27000-27999.json',
    'mpd.slice.28000-28999.json',
    'mpd.slice.29000-29999.json',
    'mpd.slice.30000-30999.json',
]

all_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        for playlist in data['playlists']:
            playlist_name = playlist['name']
            for track in playlist['tracks']:
                all_data.append({
                    "playlist_name": playlist_name,
                    "position_in_playlist": track["pos"],
                    "track_id": track['track_uri'],
                    "track_name": track['track_name'],
                    "artist_name": track['artist_name'],
                    "duration_ms": track['duration_ms'],
                    "album_name": track['album_name']
                })
                # Add and remove for DBSCAN
                '''
                if len(all_data) >= 1000:
                    break
            if len(all_data) >= 1000:
                break
        if len(all_data) >= 1000:
            break
            '''
df = pd.DataFrame(all_data)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077598 entries, 0 to 1077597
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_id              1077598 non-null  object
 3   track_name            1077598 non-null  object
 4   artist_name           1077598 non-null  object
 5   duration_ms           1077598 non-null  int64 
 6   album_name            1077598 non-null  object
dtypes: int64(2), object(5)
memory usage: 57.6+ MB


In [6]:
# Count how many playlists each track appears in
track_counts = df.groupby(['track_id', 'track_name', 'artist_name']).size().reset_index(name='count')

In [7]:
# Normalize counts to a popularity score (1-10)
min_count = track_counts['count'].min()
max_count = track_counts['count'].max()
track_counts['popularity_score'] = (
    ((track_counts['count'] - min_count) / (max_count - min_count) * 9 + 1
).astype(int))

In [8]:
# Merge popularity scores back into the original DataFrame
df_with_popularity = df.merge(
    track_counts[['track_id', 'popularity_score']],
    on='track_id',
    how='left'
)

# Popularity Score

## What Does the Score Tell You?
#### High Popularity Score (8-10):

#### Tracks with high scores are included in many playlists, indicating they are widely popular or well-liked.

#### Medium Popularity Score (4-7):

#### Tracks with medium scores are moderately popular and appear in a reasonable number of playlists.

#### Low Popularity Score (1-3):

#### Tracks with low scores are less popular and appear in only a few playlists.

In [9]:
df_with_popularity['popularity_score']

0          1
1          1
2          1
3          1
4          1
          ..
1077593    1
1077594    1
1077595    1
1077596    1
1077597    1
Name: popularity_score, Length: 1077598, dtype: int32

In [10]:
df_with_popularity[df_with_popularity['popularity_score'] == 1]

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name,popularity_score
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham,1
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham,1
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons,1
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For,1
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP,1
...,...,...,...,...,...,...,...,...
1077593,Wine,42,spotify:track:4Dno5w6kIkz9qJM9FqL61o,Medley: For Sentimental Reasons / Tenderly / A...,Natalie Cole,452746,Unforgettable: With Love,1
1077594,Wine,43,spotify:track:2PRq7qR6hT5FWfagC4wFfm,Don'cha Go 'Way Mad,Frank Sinatra,191626,Sinatra And Swingin' Brass,1
1077595,Wine,44,spotify:track:3oSFAA9uFbtokLDyLTMbX6,You Turned The Tables On Me,Quincy Jones,152026,I Dig Dancers,1
1077596,Wine,45,spotify:track:4UjeXiBihLSc7WFECGFkmJ,Knocks Me Off My Feet,Stevie Wonder,216456,Songs In The Key Of Life,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077598 entries, 0 to 1077597
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_id              1077598 non-null  object
 3   track_name            1077598 non-null  object
 4   artist_name           1077598 non-null  object
 5   duration_ms           1077598 non-null  int64 
 6   album_name            1077598 non-null  object
dtypes: int64(2), object(5)
memory usage: 57.6+ MB


In [12]:
df.head()

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP


In [13]:
track_name = df.drop_duplicates(subset=['track_name'])

In [14]:
track_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177749 entries, 0 to 1077595
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   playlist_name         177749 non-null  object
 1   position_in_playlist  177749 non-null  int64 
 2   track_id              177749 non-null  object
 3   track_name            177749 non-null  object
 4   artist_name           177749 non-null  object
 5   duration_ms           177749 non-null  int64 
 6   album_name            177749 non-null  object
dtypes: int64(2), object(5)
memory usage: 10.8+ MB


In [15]:
track_name.head(1000)

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP
...,...,...,...,...,...,...,...
1028,house/edm,63,spotify:track:6ZpR2XFuQJSHAQwg9495KZ,Mind (feat. Kai),Jack Ü,242068,Skrillex and Diplo present Jack Ü
1029,house/edm,64,spotify:track:2RpKh7kXSdO8NLrW9VQ46p,Take Ü There (feat. Kiesza),Jack Ü,210000,Skrillex and Diplo present Jack Ü
1030,house/edm,65,spotify:track:6LU4Z8hHdaHkQa2uQfpsu5,Sound Of Your Heart,Shawn Hook,199306,Analog Love
1031,house/edm,66,spotify:track:3uiQ8zYvOqOO9uBfhXdiKB,Pure Grinding,Avicii,171960,Stories


In [16]:
artists = df.drop_duplicates(subset=['artist_name'])

In [17]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45454 entries, 0 to 1077561
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         45454 non-null  object
 1   position_in_playlist  45454 non-null  int64 
 2   track_id              45454 non-null  object
 3   track_name            45454 non-null  object
 4   artist_name           45454 non-null  object
 5   duration_ms           45454 non-null  int64 
 6   album_name            45454 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.8+ MB


In [18]:
playlists = df.drop_duplicates(subset=['playlist_name'])

In [19]:
playlists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8650 entries, 0 to 1077551
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         8650 non-null   object
 1   position_in_playlist  8650 non-null   int64 
 2   track_id              8650 non-null   object
 3   track_name            8650 non-null   object
 4   artist_name           8650 non-null   object
 5   duration_ms           8650 non-null   int64 
 6   album_name            8650 non-null   object
dtypes: int64(2), object(5)
memory usage: 540.6+ KB


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Assuming df_sample is your DataFrame with 1,000 rows
# Select numerical features (e.g., duration_ms)
X = df[['duration_ms']]  # Add more features if needed

# Normalize the data (optional but recommended)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Set hyperparameters
eps = 0.8  # Adjust based on your data
min_samples = 50  # Adjust based on your data

# Train DBSCAN
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(X_scaled)

# Add cluster labels to the DataFrame
df['cluster'] = dbscan.labels_

# Display the first few rows with cluster labels
print(df.head())

# Count the number of clusters (excluding noise)
n_clusters = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noise = list(dbscan.labels_).count(-1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points: {n_noise}")

# Display the distribution of points across clusters
print(df['cluster'].value_counts())

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Assuming df_sample is your DataFrame with 1,000 rows
# Select numerical features (e.g., duration_ms)
X = df[['duration_ms']]  # Add more features if needed

# Normalize the data (optional but recommended)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Set hyperparameters
eps = 0.7  # Adjust based on your data
min_samples = 50  # Adjust based on your data

# Train DBSCAN
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(X_scaled)

# Add cluster labels to the DataFrame
df['cluster'] = dbscan.labels_

# Display the first few rows with cluster labels
print(df.head())

# Count the number of clusters (excluding noise)
n_clusters = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noise = list(dbscan.labels_).count(-1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points: {n_noise}")

# Display the distribution of points across clusters
print(df['cluster'].value_counts())

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Assuming df_sample is your DataFrame with 1,000 rows
# Select numerical features (e.g., duration_ms)
X = df[['duration_ms']]  # Add more features if needed

# Normalize the data (optional but recommended)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Set hyperparameters
eps = 0.09  # Adjust based on your data
min_samples = 50  # Adjust based on your data

# Train DBSCAN
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(X_scaled)

# Add cluster labels to the DataFrame
df['cluster'] = dbscan.labels_

# Display the first few rows with cluster labels
print(df.head())

# Count the number of clusters (excluding noise)
n_clusters = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noise = list(dbscan.labels_).count(-1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points: {n_noise}")

# Display the distribution of points across clusters
print(df['cluster'].value_counts())